In [1]:
import numpy as np
#import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D

In [2]:
cities = np.load("cities.npy", allow_pickle = True)
#cities

# 1. Ant Colony algorithm

In [7]:
# Ant Colony
# https://induraj2020.medium.com/implementation-of-ant-colony-optimization-using-python-solve-traveling-salesman-problem-9c14d3114475

In [21]:
def distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2)**2))

def ant_colony_optimization(points, n_ants, n_iterations, alpha, beta, evaporation_rate, Q):
    n_points = len(points)
    pheromone = np.ones((n_points, n_points)) # Initialise pheromones for each path between each city
    best_path = None
    best_path_length = np.inf
    
    for iteration in range(n_iterations): # Number of simulations for all ants to find path
        paths = [] # To store converged path in this iteration
        path_lengths = [] # To store the length of the coenverged path
        
        for ant in range(n_ants): # For each ant where each ant travels through all the cities
            visited = [False]*n_points # Initialise ant has not travelled to any city yet
            current_point = np.random.randint(n_points) # Randomly choose a city to visit
            visited[current_point] = True # Update visited city
            path = [current_point] # Update path of this ant 
            path_length = 0 # 
            
            while False in visited: # This while loop ensures that the ant runs through all cities
                unvisited = np.where(np.logical_not(visited))[0] # Get all non-visited cities
                probabilities = np.zeros(len(unvisited))# Initialise probability vector
                
                for i, unvisited_point in enumerate(unvisited):
                    # Updating probabilities of each unvisited city path 
                    probabilities[i] = pheromone[current_point, unvisited_point]**alpha / distance(points[current_point], points[unvisited_point])**beta
                
                # Convert to probabilities
                probabilities /= np.sum(probabilities)
                
                
                next_point = np.random.choice(unvisited, p=probabilities)# To decide next city to visit based off probability
                path.append(next_point) # Add next city to path
                path_length += distance(points[current_point], points[next_point]) # Update path length
                visited[next_point] = True # Update that the city has been visited
                current_point = next_point # Update current position
            
            paths.append(path)# Updates the whole path the ant took, each path is a list of numbers
            path_lengths.append(path_length)# Updates the best 
            
            if path_length < best_path_length: # Update best path and best path length
                best_path = path
                best_path_length = path_length
        
        pheromone *= evaporation_rate # This assumes that pheromones changes after each ants journey
        
        for path, path_length in zip(paths, path_lengths):
            for i in range(n_points-1):
                pheromone[path[i], path[i+1]] += Q/path_length
            pheromone[path[-1], path[0]] += Q/path_length
            
            
    return(best_path,best_path_length)
    
# Example usage:
#points = np.random.rand(10, 3) # Generate 10 random 3D points

#ant_colony_optimization(points, n_ants=10, n_iterations=100, alpha=1, beta=1, evaporation_rate=0.5, Q=1)
#cities

In [6]:
# Random parameters, this took 30min to run
best_path,best_length = ant_colony_optimization(cities, n_ants=10, n_iterations=10, alpha=1, beta=1, evaporation_rate=0.5, Q=1)

In [9]:
import pickle
with open("ant_colony_base_path", "wb") as fp:   #Pickling
    pickle.dump(best_path, fp)
#best_path
#best_length # 318.36

In [10]:
# Parameters decided from paper, ran it overnight
# http://article.nadiapub.com/IJUNESST/vol7_no4/16.pdf 
best_path_s,best_length_s = ant_colony_optimization(cities, n_ants=250, n_iterations=10, alpha=1, beta=5, evaporation_rate=0.5, Q=1)

In [16]:
#with open("ant_colony_tuned_path", "wb") as fp:   #Pickling
#    pickle.dump(best_path_s, fp)
    
# best_length_s # 28.377
#best_path_s[]
#distance(cities[393],cities[0])

0.5882100091453807

# 1.1 Extensions of Ant Colony
1. Using MST distance as stopping condition for number of iterations
2. Changing the way probabilites of each path is derived for each path
## 1. Using MST as stopping condition

In [38]:
# Kruskal's Algorithm to get MST length
# https://www.pythonpool.com/kruskals-algorithm-python/
class Graph:
    def __init__(self, vertex):
        self.V = vertex
        self.graph = []
 
    def add_edge(self, u, v, w):
        self.graph.append([u, v, w])
 
 
    def search(self, parent, i):
        if parent[i] == i:
            return i
        return self.search(parent, parent[i])
 
    def apply_union(self, parent, rank, x, y):
        xroot = self.search(parent, x)
        yroot = self.search(parent, y)
        if rank[xroot] < rank[yroot]:
            parent[xroot] = yroot
        elif rank[xroot] > rank[yroot]:
            parent[yroot] = xroot
        else:
            parent[yroot] = xroot
            rank[xroot] += 1
 
  
    def kruskal(self):
        result = []
        i, e = 0, 0
        self.graph = sorted(self.graph, key=lambda item: item[2])
        parent = []
        rank = []
        for node in range(self.V):
            parent.append(node)
            rank.append(0)
        while e < self.V - 1:
            u, v, w = self.graph[i]
            i = i + 1
            x = self.search(parent, u)
            y = self.search(parent, v)
            if x != y:
                e = e + 1
                result.append([u, v, w])
                self.apply_union(parent, rank, x, y)
        total = 0
        for u, v, weight in result:
            #print("Edge:",u, v,end =" ")
            #print("-",weight)
            total += weight
        print(total)
        return(total,result)
#g = Graph(5)
#g.add_edge(0, 1, 8)
#g.add_edge(0, 2, 5)
#g.add_edge(1, 2, 9)
#g.add_edge(1, 3, 11)
#g.add_edge(2, 3, 15)
#g.add_edge(2, 4, 10)
#g.add_edge(3, 4, 7)
#g.kruskal()

In [39]:
dist_matrix = [[distance(c1, c2) for c2 in cities] for c1 in cities]
g = Graph(1000)
for i in range(0,1000):
    for j in range(i,1000):
        g.add_edge(i,j,dist_matrix[i][j])
min_length, path = g.kruskal()

In [22]:
def distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2)**2))

def ant_colony_optimization_MST(points, n_ants, n_iterations, alpha, beta, evaporation_rate, Q,start_point):
    n_points = len(points)
    pheromone = np.ones((n_points, n_points)) # Initialise pheromones for each path between each city
    best_path = None
    best_path_length = np.inf
    
    for iteration in range(n_iterations): # Number of simulations for all ants to find path
        paths = [] # To store converged path in this iteration
        path_lengths = [] # To store the length of the coenverged path
        
        for ant in range(n_ants): # For each ant where each ant travels through all the cities
            visited = [False]*n_points # Initialise ant has not travelled to any city yet
            current_point = start_point
            visited[current_point] = True # Update visited city
            path = [current_point] # Update path of this ant 
            path_length = 0 # 
            
            while False in visited: # This while loop ensures that the ant runs through all cities
                unvisited = np.where(np.logical_not(visited))[0] # Get all non-visited cities
                probabilities = np.zeros(len(unvisited))# Initialise probability vector
                
                for i, unvisited_point in enumerate(unvisited):
                    # Updating probabilities of each unvisited city path 
                    probabilities[i] = pheromone[current_point, unvisited_point]**alpha / distance(points[current_point], points[unvisited_point])**beta
                
                # Convert to probabilities
                probabilities /= np.sum(probabilities)
                
                
                next_point = np.random.choice(unvisited, p=probabilities)# To decide next city to visit based off probability
                path.append(next_point) # Add next city to path
                path_length += distance(points[current_point], points[next_point]) # Update path length
                visited[next_point] = True # Update that the city has been visited
                current_point = next_point # Update current position
            
            paths.append(path)# Updates the whole path the ant took, each path is a list of numbers
            path_lengths.append(path_length)# Updates the best 
            
            if path_length < best_path_length: # Update best path and best path length
                best_path = path
                best_path_length = path_length
        pheromone *= evaporation_rate # This assumes that pheromones changes after each ants journey
        
        for path, path_length in zip(paths, path_lengths):
            for i in range(n_points-1):
                pheromone[path[i], path[i+1]] += Q/path_length
            pheromone[path[-1], path[0]] += Q/path_length
            
        if (best_path_length <= 25): # This is to check for the path if the length is already the best we can get by MST
            break
            
    return(best_path,best_path_length)
    
# Example usage:
#points = np.random.rand(10, 3) # Generate 10 random 3D points

#ant_colony_optimization(points, n_ants=10, n_iterations=100, alpha=1, beta=1, evaporation_rate=0.5, Q=1)
#cities

In [25]:
# best_path_s_1,best_path_length_s_2 = ant_colony_optimization_MST(cities, n_ants=100, n_iterations=100, alpha=1, beta=5, evaporation_rate=0.5, Q=1,start_point = 3)
# yielded similar results as the tuned base ant colony algo

# 2. Extensions
## 2.2. Ant colony system  
### Differences
#### 1. State transition rule is different, ie the probability distribution of which path the ant takes at each step is different
#### 2. Global updating rule is only applied to the path of the best ant, ie the pheromone evaporation rule, note that the paper stated that using the local best as opposed to the global best yield minimal differences
#### 3. Local updating rule stays the same

In [3]:
def distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2)**2))

def ACS(points, n_ants, n_iterations, alpha, beta, evaporation_rate, Q,start_point,q0):
    n_points = len(points)
    pheromone = np.ones((n_points, n_points)) 
    best_path = None
    best_path_length = np.inf
    for iteration in range(n_iterations):
        paths = [] 
        path_lengths = [] 
        
        for ant in range(n_ants): 
            visited = [False]*n_points 
            current_point = start_point
            visited[current_point] = True 
            path = [current_point] 
            path_length = 0 # 
            
            while False in visited: 
                unvisited = np.where(np.logical_not(visited))[0] 
                probabilities = np.zeros(len(unvisited))
                storage = np.zeros(len(unvisited))
                for i, unvisited_point in enumerate(unvisited):
                    probabilities[i] = pheromone[current_point, unvisited_point]**alpha / distance(points[current_point], points[unvisited_point])**beta
                    storage[i] = pheromone[current_point, unvisited_point]**alpha / distance(points[current_point], points[unvisited_point])**beta
                probabilities /= np.sum(probabilities)
                
                # DIFFERENCE 1: STATE TRANSITION RULE
                decider = np.random.uniform(0,1,1)
                
                if (decider>q0):
                    next_point = np.random.choice(unvisited, p=probabilities)
                else:
                    next_point = np.argmax(storage)
                
                path.append(next_point)
                path_length += distance(points[current_point], points[next_point]) 
                visited[next_point] = True 
                current_point = next_point 
            
            paths.append(path)
            path_lengths.append(path_length)
            
            if path_length < best_path_length: 
                best_path = path
                best_path_length = path_length
                
        # DIFFERENCE 2: CHANGE IN GLOBAL UPDATING RULE WHERE BEST PATH GETS UPDATED DIFFERENTLY
        pheromone *= evaporation_rate
        for i in range(0,n_points-1):
            pheromone[best_path[i]][best_path[i+1]] += evaporation_rate * (distance(points[best_path[i]],points[best_path[i+1]]))**(-1)
        
        # DIFFERENCE 3: CHANGE IN LOCAL UPDATING RULE
        for path, path_length in zip(paths, path_lengths): # This is to add the pheromone from current ant
            for i in range(n_points-1):
                pheromone[path[i], path[i+1]] += Q/path_length
            pheromone[path[-1], path[0]] += Q/path_length
            
            
    return(best_path,best_path_length)
    


In [4]:
best_path_ACS,best_length_ACS = ACS(cities, n_ants=250, n_iterations=10, alpha=1, beta=5, evaporation_rate=0.5, Q=1,start_point = 1,q0 = 0.3)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: divide by zero encountered in double_scalars


In [6]:
#This took 12 hrs to run
#best_path_ACS # 184.59
#best_length_ACS with tuned parameters from original ACO

184.5945289180284

In [8]:
import pickle
with open("ACS_path", "wb") as fp:   #Pickling
    pickle.dump(best_path_ACS, fp)